In [1]:
import pyhdf.SD as sds
import pandas as pd
import os
from datetime import datetime
from IPython.display import clear_output

# 7,5 km
#Manaus_EMBRAPA -2.954minlat -2.828maxlat -60.033minlong -59.907maxlong
#Rio Branco (9.957S, 67.869W) -10.02minlat -9.894maxlat -67.932minlong -67.806maxlong
#Ji_Parana_SE (10.934S, 61.852W) -10.997minlat -10.871maxlat -61.915minlong -61.789maxlong


min_lat = -2.954
max_lat = -2.828
min_long = -60.033
max_long = -59.907

folder_path = "teste"

files = [file for file in os.listdir(folder_path) if file.endswith('.hdf')]
n_files = len(files)
n_files

11

In [2]:
output_values = []
counter = 0
for file in files:
    file_path = os.path.join(folder_path, file)

    hdf = sds.SD(file_path, sds.SDC.READ)

    AOD = hdf.select('Optical_Depth_Land_And_Ocean')[:]
    Longitude = hdf.select('Longitude')[:]
    Latitude = hdf.select('Latitude')[:]

    df = pd.DataFrame()
    df['AOD'] = AOD.flatten()
    df['Longitude'] = Longitude.flatten()
    df['Latitude'] = Latitude.flatten()

    filtered_df = df.copy()
    filtered_df = df[df['AOD'] != -9999]
    filtered_df = filtered_df.query('Latitude >= @min_lat and Latitude <= @max_lat and Longitude >= @min_long and Longitude <= @max_long')

    mean_aod = filtered_df.AOD.mean() * 0.001
    output_values.append(mean_aod)

    clear_output()
    counter+=1
    print(f"{counter} of {n_files} done.")


11 of 11 done.


In [3]:
def julian_with_year_to_date(julian_date_with_year):
    return datetime.strptime(str(julian_date_with_year), '%Y%j').date()

dates = []

for f in files:
    year = f[10:14]
    day = f[14:17]
    dates.append(julian_with_year_to_date(f'{year}{day}'))

In [4]:
final_df = pd.DataFrame()

final_df['date'] = dates
final_df['AOD'] = output_values

final_df.to_csv('MOD043K_teste.csv')

In [5]:
df = pd.read_csv("MOD043K_teste.csv")
df.tail()

,Unnamed: 0,date,AOD
6,6,2010-02-04,NaN
7,7,2010-02-04,NaN
8,8,2010-02-05,NaN
9,9,2010-02-04,NaN
10,10,2010-02-05,NaN
